In [1]:
import pandas as pd
import torch.nn as nn
import torch
import torch.optim as optim
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
from tqdm.notebook import tqdm

In [2]:
#3 layer MLP with one hidden layer
class MultilayerPerceptron(nn.Module):
    #define network
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) # input layer -> hidden layer
        self.tanh = nn.Tanh()
        self.fc2 = nn.Linear(hidden_size, output_size)

    #define forward function
    def forward(self, x):
        out = self.fc1(x)
        out = self.tanh(out)
        out = self.fc2(out)
        return out
    
    

In [30]:
def training(model, dataloader, func_loss, opt, num_epochs, experiment_name):
    model.train()
    best_model = None
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        for i, batched_data in enumerate(dataloader): 
            inputs, targets = batched_data
#             print("inputs: " + str(inputs.shape))
#             print("target: " + str(targets.shape))
            
            output_prob = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)
            loss = func_loss(output_prob.squeeze(), targets)
            
            # Backward and optimize
            loss.backward() #performing back prop
            opt.step() #updating the weights
            opt.zero_grad() #reset gradients
            
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(output_prob == targets.data)
        
        epoch_loss = running_loss / 100
        epoch_acc = running_corrects.double() / 100
        
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model.state_dict())
            torch.save(best_model, os.path.join(experiment_name,
                                                'best_model.pt'))
            
        if epoch % 5 == 0:
            if epoch_acc > 50:
                print("accuracy for epoch: " + str(epoch_acc))
            print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))
            
    print('Epoch: ' + str(epoch) + ' Final loss is: ' + str(loss.item()))

In [7]:
# df = pd.read_csv('../Data/PGH/DemandPrediction/ml_df.csv')
# df = df.drop(columns=['Unnamed: 0', 'median_hh_income','mean_hh_income','in_labor_force', 'civ_labor_force', 
#                       'employed','unemployed', 'perc_employed', 'perc_unemployed','own_alone', 
#                       'carpool', 'public_transit', 'walked', 'other', 'wfh','mean_travel_time_to_work', 
#                       'perc_public', 'perc_alone', 'perc_walk','perc_other', '2018_outflow', '2019_outflow', '2020_outflow'])
df = pd.read_csv('../Data/PGH/outflow_pgh_monthly.csv')
df = df.drop(columns=['station_id', 'tractce10'])

for row in range(len(df)):
    for col in df.columns:
        if df.loc[row][col] == '-' or pd.isna(df.loc[row][col]):
            df.at[row,col] = 0
            
df = df.fillna(0)
            

X = torch.from_numpy(df.iloc[:, 0:67].to_numpy()).type(torch.float)
y = torch.from_numpy(df.iloc[:, 67].to_numpy()).type(torch.float)

# mean = torch.mean(X, axis=0)
# stdev = torch.std(X, axis=0)
# X = (X - mean)/stdev
print(len(X))

dataset = TensorDataset(X, y)

train_ds, test_ds = random_split(dataset, [70, 30])
batch_size=1
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
test_loader = DataLoader(test_ds, batch_size, shuffle=True)

100


In [21]:

input_size = len(df.iloc[:,0:67].columns)
output_size = 1
input_size,output_size

(67, 1)

In [14]:
def evaluation(model, dataloader, func_loss):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        right, running_total = 0, 0

        for i, batched_data in enumerate(dataloader):

            #moving tensors to the GPU
            inputs, targets = batched_data #grab the batch of images as tensors and grab the batch of labels as tensors

            output_probs = model(inputs) #pass images through model to get probability for each class (1 x 10 dim)

            loss = func_loss(output_probs.squeeze(), targets)
            print("loss: " + str(loss))
            total_loss += loss
 
        print('Total Loss L1: ' + str(total_loss))

In [31]:
# model = MultilayerPerceptron(input_size, 150, output_size)
model = nn.Sequential(nn.Linear(input_size, 5), nn.ELU(),
                     nn.Linear(5, 9), nn.ELU(),
                     nn.Linear(9, 9), nn.ELU(),
                     nn.Linear(9, 5), nn.ELU(),
                     nn.Linear(5, 1))
epochs = 500
learning_rate = 1e-4
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=1e-4, momentum=0.99)
training(model, train_loader, nn.L1Loss(), optimizer, epochs, "test1")

Epoch: 0 Final loss is: 13.293106079101562
Epoch: 5 Final loss is: 3.078004837036133
Epoch: 10 Final loss is: 8.035240173339844
Epoch: 15 Final loss is: 4.980989456176758
Epoch: 20 Final loss is: 10.992538452148438
Epoch: 25 Final loss is: 4.951473236083984
Epoch: 30 Final loss is: 6.953508377075195
Epoch: 35 Final loss is: 7.886628150939941
Epoch: 40 Final loss is: 31.9925537109375
Epoch: 45 Final loss is: 5.9711103439331055
Epoch: 50 Final loss is: 7.93890380859375
Epoch: 55 Final loss is: 5.177516937255859
Epoch: 60 Final loss is: 5.021839141845703
Epoch: 65 Final loss is: 2.0767364501953125
Epoch: 70 Final loss is: 0.010975837707519531
Epoch: 75 Final loss is: 3.9782304763793945
Epoch: 80 Final loss is: 0.9392175674438477
Epoch: 85 Final loss is: 43.27016067504883
Epoch: 90 Final loss is: 10.909214973449707
Epoch: 95 Final loss is: 4.886701583862305
Epoch: 100 Final loss is: 11.926839828491211
Epoch: 105 Final loss is: 9.909698486328125
Epoch: 110 Final loss is: 3.493607521057129
E

In [24]:
evaluation(model, test_loader, nn.L1Loss())

loss: tensor(9.0703)
loss: tensor(2.9297)
loss: tensor(0.0703)
loss: tensor(9.9297)
loss: tensor(4.9297)
loss: tensor(2.0703)
loss: tensor(6.9297)
loss: tensor(4.0703)
loss: tensor(10.9297)
loss: tensor(13.9297)
loss: tensor(6.9297)
loss: tensor(0.9297)
loss: tensor(15.0703)
loss: tensor(11.9297)
loss: tensor(26.0703)
loss: tensor(10.9297)
loss: tensor(2.9297)
loss: tensor(42.0703)
loss: tensor(13.9297)
loss: tensor(26.0703)
loss: tensor(6.9297)
loss: tensor(127.0703)
loss: tensor(3.9297)
loss: tensor(9.9297)
loss: tensor(9.9297)
loss: tensor(3.0703)
loss: tensor(9.9297)
loss: tensor(12.9297)
loss: tensor(5.9297)
loss: tensor(2.0703)
Total Loss L1: tensor(413.4374)
